In [1]:
pip show kfre # inspect the library

Name: kfre
Version: 0.1.0a1
Summary: A Python library for kidney failure risk estimation using Tangri's KFRE model
Home-page: https://github.com/lshpaner/kfre
Author: Leonid Shpaner
Author-email: lshpaner@ucla.edu
License: 
Location: c:\users\lshpaner\anaconda3\envs\custom_conda\lib\site-packages
Requires: numpy, pandas
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from kfre import RiskPredictor

# Sample data
## Note:
data = pd.DataFrame(
    {
        "Age": [65, 70],
        "Gender": ["male", "female"],
        "eGFR": [20, 50],
        "Urine_Albumin_Creatinine_Ratio": [30, 35],  # in mg/g
        "Region": ["North America", "Other"],
        "Diabetes": [1, 0],
        "Hypertension": [1, 1],
    }
)


# Column mapping
columns = {
    "age": "Age",
    "sex": "Gender",
    "eGFR": "eGFR",
    "uACR": "Urine_Albumin_Creatinine_Ratio",
    "region": "Region",
    "dm": "Diabetes",
    "htn": "Hypertension",
}

# Initialize the predictor
predictor = RiskPredictor(data, columns)

# Predict CKD risk for 2 years using basic 4-variable model
data["2_Year_Risk"] = predictor.predict_kfre(
    years=2, is_north_american=True, num_vars=4
)

# Predict CKD risk for 5 years using basic 4-variable model
data["5_Year_Risk"] = predictor.predict_kfre(
    years=5, is_north_american=True, num_vars=4
)

print(data[["Age", "Gender", "2_Year_Risk", "5_Year_Risk"]])

   Age  Gender  2_Year_Risk  5_Year_Risk
0   65    male     0.083477     0.238253
1   70  female     0.002315     0.007209
